In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from psycopg2.extras import execute_batch
from sql_queries import summary_table_insert
import urllib.parse

In [7]:
def get_table_columns(table_element):
    header_container = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="tableHeaderDiv", unique_element=True)
    header_elements = wait_for_element(source=header_container, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in header_elements]
    
    return header_names

def get_table_rows(table_element):
    table_rows = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    
    return rows

In [8]:
class Scraper:
    def __init__(self):
        self.driver = None
        self.conn = None
        self.cur = None
        self.table_data = None
        self.table_headers = None
        self.table_insert = ""
    
    def open_driver(self):
        self.driver = open_driver()
    
    def close_driver(self):
        if self.driver:
            close_driver(self.driver)
            
    def db_connect(self):
        self.conn, self.cur = db_connect()
    
    def close_db_connect(self):
        if self.conn and self.cur:
            close_db_connect(self.conn, self.cur)
            
    def extract(self, url):
        """sends requests to the url and scrapes the table elements"""
        self.driver.get(url)
        
        # scrape the page's main table
        root_element = wait_for_element(source=self.driver, search_by=By.ID, target="root", unique_element=True)
        data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)
        
        header_elements = get_table_columns(data_table)
        row_elements = get_table_rows(data_table)
        
        return header_elements, row_elements
    

    def load(self, data_rows):
        execute_batch(self.cur, self.table_insert, data_rows)

In [9]:
class SummaryScraper(Scraper):
    def __init__(self):
        super().__init__()
        self.table_insert = summary_table_insert
        self.base_url = "https://www.nhl.com/stats/teams?"
    
    def build_url(self):
        params = {
            "aggregate": 0,
            "reportType": "game",
            "seasonFrom": 20212022,
            "seasonTo": 20212022,
            "dateFromSeason": [],
            "gameType": 2,
            "page": 0,
            "pageSize": 100}
        
    def transform(self, headers, row_elements):
        table_data = []
        
        for row in row_elements:
            row_cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
            row_values = [cell.text for cell in row_cells]
            
            values_map = list(zip(headers, row_values))
            map_dict = parse_record(values_map)
            
            self.cur.execute("SELECT abbreviation FROM teams WHERE team = %s;", (map_dict["Team"],))
            team_abbreviation = self.cur.fetchone()[0]
            
            ordered_data_list = [
                team_abbreviation,
                "2021/22",
                map_dict["game_date"],
                map_dict["home_game"],
                map_dict["opponent"],
                map_dict["W"],
                map_dict["L"],
                map_dict["T"],
                map_dict["OT"],
                map_dict["P"],
                map_dict["P%"],
                map_dict["RW"],
                map_dict["ROW"],
                map_dict["S/O Win"],
                map_dict["GF"],
                map_dict["GA"],
                map_dict["PP%"],
                map_dict["PK%"],
                map_dict["Net PP%"],
                map_dict["Net PK%"],
                map_dict["Shots/GP"],
                map_dict["GA/GP"],
                map_dict["FOW%"]
            ]
        
            table_data.append(ordered_data_list)
        
        return table_data
    
    def etl(self, url):
        # the chrome driver and database cursor are used in multiple scripts
        self.open_driver()
        self.db_connect()
        
        table_headers, table_rows = self.extract(url)
        parsed_data = self.transform(table_headers, table_rows)        
        
        # TODO: execute batch loading
        self.load(parsed_data)
        
        # close the driver and the cursor
        self.close_driver()
        self.close_db_connect()
        
        return parsed_data

In [10]:
scraper = SummaryScraper()

In [11]:
summary_table_data = scraper.etl("https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20212022&dateFromSeason&gameType=2&page=0&pageSize=100")

In [14]:
len(summary_table_data)

100

In [11]:
url = 'https://example.com/somepage/?'
params = {'var1': 'some data', 'var2': 1337}
print(url + urllib.parse.urlencode(params))

https://example.com/somepage/?var1=some+data&var2=1337


In [15]:
url = "https://www.nhl.com/stats/teams?"
# params = {"aggregate": 0, "reportType": "game", "seasonFrom": 20212022, "seasonTo": 20212022, "gameType": 2}
# print(url + urllib.parse.urlencode(params))

In [22]:
# https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20212022&dateFromSeason&gameType=2&page=0&pageSize=100
params = {"aggregate": 0, "reportType": "game", "seasonFrom": 20212022, "seasonTo": 20212022, "dateFromSeason": [], "gameType": 2, "page": 0, "pageSize": 100}

In [23]:
print(url + urllib.parse.urlencode(params))

https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20212022&dateFromSeason=%5B%5D&gameType=2&page=0&pageSize=100
